In [ ]:
# install gitPython
import os, sys, time
!pip install gitPython

# clone repository
import git
!git clone https://github.com/sungbinlim/RLclass.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
Cloning into 'RLclass'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 54 (delta 13), reused 41 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), 10.22 KiB | 615.00 KiB/s, done.


In [ ]:
# append package address
%cd /content/RLclass/STAT436/grid_world/
sys.path.append("/content/RLclass/STAT436/grid_world/") 
from grid_world import *

/content/RLclass/STAT436/grid_world


MC 로 Dynamics사용하지 않고, Environment만 가지고 Dyanmics를 구현할것임.

Policy Iteration
1. Policy Evaluation of 'state-value fucntion' via `mc.eval` 

  we need to collect trajectory (history) 

  agent do this. from `grid_world.py` - `class Agent:` 
  (reset `self.states` by `def reset`)

  from `def play`, append `self.states` that contains state, action to `self.history`

2. Policy Improvement

  

In [ ]:
import numpy as np

def compute_position(state):
    # state(12) -> array(3, 4)
    return np.array([state // 4, state % 4])

def reverse_position(array):
    # array(3, 4) -> state(12)
    return array[0] * 4 + array[1]

# action value input dim : 12,4
def create_value_memory(dim=(12, 4)):
    value_memory = np.zeros(dim)
    return value_memory

def action_to_index(action):
    # up, left, right, down
    if action == 'up':
        return 0
    if action == 'left':
        return 1
    if action == 'right':
        return 2
    if action == 'down':
        return 3
    else:
        raise ValueError('not proper action')

def transform_trajectory_memory(trajectory):
    # trajectory -> {(state, action)}
    trajectory_ = [(reverse_position(pair[0]), action_to_index(pair[1])) for pair in trajectory]

    return trajectory_

# 이 부분
def mc_eval(history, reward_stat, gamma, update=0.99):
    """
    input: history
    output: value estimation
    """
    value_memory = create_value_memory()
    
    for trajectory, reward in zip(history, reward_stat): # python에서는 묶어줘야함
        # transform_trajectory memory 로 (3,2)같이 되어있는 걸 각각의 state, action의 N으로 바꿔줌
        trajectory_ = transform_trajectory_memory(trajectory)
        T = len(trajectory_) - 1
        G = reward

        tmp = np.zeros.like(value_memory)
        tmp[trajectory_[T][0], trajectory_[T][1]] = G
        # `trajectory_[T][0]` T번째 종결시점에서 첫번째 정보 : state
        # `trajectory_[T][1]` T번째 종결시점에서 두번째 정보 : action
        # MC에서 Agent 가 종결시점 T에 도달했을때 state value, action value를 각각 집어넣어줌
        # T시점에서는 앞에 state와 action이 어떤거였는지 모름
        # backward 방식으로 state, action 을 T시점 state value, action value로 converge해줄것임.
        
        for i in range(1, T+1, 1):
            G = gamma * G
            tmp[trajectory_[T-t][0], trajectory_[T-t][1]] = G
            # backward로 state-value, action value를 update
        # tmp는 단일 에피소드에 대해 새로 얻은 state-value, action-value 값들

        """
        incremental update
        새로운 정보(tmp)는 0.99만큼, 맨처음 들어왔던 정보(value_memory)는 0.01만큼 믿어주세요
        """
        value_memory = value_memory + update * (tmp-value_memory)
        
    return value_memory

# from Policy Iteration
def one_hot(scalar, dim):
    vec = np.zeros(dim)
    vec[scalar] = 1
    return vec

"""
이부분 숙제 : greedy policy 짤때, maximum 하게 만들어주는 policy가 unique하지 않으면 tie라는 개념을 통해 분할해줘야함
"""
def greedy_action(array, dim):


    return vec

def argmax(vec, tie=True):
        

# update policy w/ greedy policy
def update_policy(policy, action_value):

    greedy_policy = np.zeros_like(policy)

    for state in range(12):

        action = argmax(action_value[state, :])
        action = greedy_action(action, 4)
        greedy_policy[state] = action

    return greedy_policy

# Policy Improvement w/ MC
def mc_policy_iteration(pi_init, agent, gamma, eps=1e-8, play_num=100, epsilon=None):

    # call policy eval
    pi = pi_init
    agent_ = agent(pi_init)
    epsilon_init = epsilon

    """이부분"""
    # grid_world.py class Agent 의 def play는 reward_stat, history를 return함
    history, reward_stat ,success_rate = agent_.play(play_num)
    action_value = mc_eval(history, reward_stat, gamma)

    advances = np.inf
    n_it = 0

    while advances > eps or n_it <= 2:
        """우리가 구한 action_value를 기반으로 policy를 update해줘야 함"""

        pi_new = update_policy(pi, action_value) # new policy

        """exploration을 위해서는 epsilon soft를 적용해줘야함"""
        agent_ = agent(pi_new, ) # new policy로 episode를 모아라, epsilon soft하게 # `grid_world.py`에서 class Agent 에 def Action에 구현되어있음
        history, reward_stat, success_rate = agent_.play(play_num)
        action_value_new = mc_eval(history, reward_stat, gamma)

        advances = action_value_new - action_value
        advances = np.sum(np.abs(advances))

        """save pi as pi_new"""
        pi = pi_new
        action_value = action_value_new
        n_it += 1
        epsilon = epsilon_init / n_it # inital epsilon으로 해줘야지 exploration 의 concept을 구현할 수 있음.

        if n_it % 10 == 0:
            print("Iteration: {}, Success rate:{} %, Error: {}, eps: {}".format(play_num * n_it, success_rate * 100, advances, epsilon))

    print("Monte-Carlo Policy Iteration converged. (Iteration={}, Error={})".format(play_num * n_it, advances))

    return pi_new, action_value_new

IndentationError: ignored

In [ ]:
import time


gamma = 0.99
# random policy function
pi = np.array([0.25, 0.25, 0.25, 0.25]) #up, left, right, down
pi = np.reshape(np.tile(pi, 12), (12, 4))

print("\nUpdating Policy via Policy Iteration w/ Monte-Carlo")
start_time = time.time()
pi_new, action_value_new = mc_policy_iteration(pi, Agent, gamma, play_num=100, epsilon=0.1)
end_time = time.time()
computation_time = end_time - start_time
print("Wall-clock time for Policy Iteration: {} sec\n".format(np.round(computation_time, 4)))

print("Let's run grid world!")
agent = Agent(pi_new)
success_rate = agent.play(100, stat=True)
# agent.show_policy()
print("action value:\n {}".format(np.round(action_value_new, 3)))
print("Success rate:{} %".format(success_rate * 100))
